# ``UNet`` demo

The UNet is a classic go-to for semantic segmentation. To demonstrate it, we can use the VOCSegmentation dataset, which we can access through ``torchvision``.

In [ ]:
from torchvision.datasets import VOCSegmentation